In [68]:
import numpy as np
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import classification_report
from collections import Counter
import pandas as pd
from sklearn.model_selection import StratifiedKFold,  cross_val_score
from sklearn.metrics import make_scorer, fbeta_score

In [69]:
# Lade Datensatz
train_data_loaded = pd.read_csv('../../data/train_data_2024-08-01.csv')
X = train_data_loaded.drop(columns=['UKATEGORIE'])
y = train_data_loaded['UKATEGORIE']


In [70]:
# StratifiedKFold initialisieren
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Variablen für Ergebnisse
fbetas_rf = []
fbetas_random = []
fbetas_mehrheit = []

In [71]:
beta = 2
fbeta_scorer= make_scorer(fbeta_score, beta=beta)

In [72]:
model_logreg = LogisticRegression(C=1, max_iter=1700, penalty='l2', solver='liblinear', tol=0.0001, random_state=42, class_weight = {0:1, 1: 9})

logreg_scores = cross_val_score(model_logreg, X, y, cv=skf, scoring=fbeta_scorer)
print(f'LogReg F betas: ', logreg_scores)
print(f'LogReg F-beta Score (mean): {logreg_scores.mean()}')
print(f'LogReg F-beta Score (std): {logreg_scores.std()}')

counter = 1
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Drucke die ersten 10 values von y_train
    print("Die ersten 20 Werte von y_train:               ", y_train.values[:20])
    # Drucke die ersten 10 values von y_test
    print("Die ersten 20 Werte von y_test:                ", y_test.values[:20])
    
    # Random Forest Modell trainieren
    model_logreg.fit(X_train, y_train)
    y_pred_rf = model_logreg.predict(X_test)
    print("Ersten 20 Vorhersagen mit dem Modell:          ", y_pred_rf[:20])
    
    # Zufällige Vorhersagen basierend auf den Klassenwahrscheinlichkeiten erstellen
    class_counts = Counter(y_test)
    total_samples = len(y_test)
    class_probabilities = {cls: count / total_samples for cls, count in class_counts.items()}
    
    np.random.seed(42)
    y_pred_random_weighted = np.random.choice(
        list(class_probabilities.keys()),
        size=y_test.shape,
        p=list(class_probabilities.values())
    )
    
    # Drucke die ersten 10 Werte
    print("Ersten 20 Vorhersagen mit dem _random_weighted:", y_pred_random_weighted[:20])
    
    # Leistung der gewichteten zufälligen Vorhersagen bewerten
    fbeta_random_weighted = fbeta_score(y_test, y_pred_random_weighted, beta=beta, average='weighted')
  
    fbetas_random.append(fbeta_random_weighted)
    
    # Immer Mehrheitsklasse vorhersagen
    y_pred_majority = np.full(y_test.shape, y_train.mode().iloc[0])
    
    # Drucke die ersten 10 Werte
    print("Ersten 10 Vorhersagen mit dem y_pred_majority: ", y_pred_majority[:10])
    
    # Leistung der Vorhersagen der Mehrheitsklasse bewerten
    fbeta_mehrheit = fbeta_score(y_test, y_pred_majority, beta=beta, average='weighted')
    
    print(f"Fold Nummer {counter}: fbeta_random_weighted - Score ist {fbeta_random_weighted}")
    print(f"Fold Nummer {counter}: fbeta_mehrheit - Score ist {fbeta_mehrheit}")
    fbetas_mehrheit.append(fbeta_mehrheit)
    
    counter += 1
    
    

LogReg F betas:  [0.48882557 0.48079506 0.48864846 0.47506045 0.47869495]
LogReg F-beta Score (mean): 0.48240489744404214
LogReg F-beta Score (std): 0.005486407299560132
Die ersten 20 Werte von y_train:                [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1]
Die ersten 20 Werte von y_test:                 [0 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0]
Ersten 20 Vorhersagen mit dem Modell:           [1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 0]
Ersten 20 Vorhersagen mit dem _random_weighted: [0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0]
Ersten 10 Vorhersagen mit dem y_pred_majority:  [0 0 0 0 0 0 0 0 0 0]
Fold Nummer 1: fbeta_random_weighted - Score ist 0.7465515104431596
Fold Nummer 1: fbeta_mehrheit - Score ist 0.8168620906598427
Die ersten 20 Werte von y_train:                [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0]
Die ersten 20 Werte von y_test:                 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0]
Ersten 20 Vorhersagen mit dem Modell:           [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
E

In [73]:
# Durchschnittliche Genauigkeiten berechnen
mean_scores = np.mean(logreg_scores)
mean_scores_random = np.mean(fbetas_random)
mean_scores_mehrheit = np.mean(fbetas_mehrheit)

In [74]:
print(f"LogReg Mean score: {mean_scores}")
print(f"Weighted Random Prediction Mean score: {mean_scores_random}")
print(f"Weighted Mehrheit Prediction Mean score: {mean_scores_mehrheit}")


LogReg Mean score: 0.48240489744404214
Weighted Random Prediction Mean score: 0.7426462385123551
Weighted Mehrheit Prediction Mean score: 0.8167984428179098


In [75]:

# Klassifikationsberichte für den letzten Fold ausgeben
print("Random Forest Classification Report (Last Fold):")
print(classification_report(y_test, y_pred_rf))

print("Weighted Random Prediction Classification Report (Last Fold):")
print(classification_report(y_test, y_pred_random_weighted))

Random Forest Classification Report (Last Fold):
              precision    recall  f1-score   support

           0       0.91      0.23      0.37      6787
           1       0.17      0.87      0.29      1231

    accuracy                           0.33      8018
   macro avg       0.54      0.55      0.33      8018
weighted avg       0.80      0.33      0.36      8018

Weighted Random Prediction Classification Report (Last Fold):
              precision    recall  f1-score   support

           0       0.85      0.85      0.85      6787
           1       0.16      0.15      0.15      1231

    accuracy                           0.74      8018
   macro avg       0.50      0.50      0.50      8018
weighted avg       0.74      0.74      0.74      8018

